In [187]:
import pandas as pd
import numpy as np

df = pd.read_csv('D:\\coding Stuff\\DataSets\\marketing_campaign.csv', sep = '\t')

In [188]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [189]:
df.shape

(2240, 29)

Now that we have the data read, lets look at some basic statistics of that data. 

In [190]:
df.describe()

,ID,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
count,2240.000000,2240.000000,2216.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,...,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.0,2240.0,2240.000000
mean,5592.159821,1968.805804,52247.251354,0.444196,0.506250,49.109375,303.935714,26.302232,166.950000,37.525446,...,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393,0.009375,3.0,11.0,0.149107
std,3246.662198,11.984069,25173.076661,0.538398,0.544538,28.962453,336.597393,39.773434,225.715373,54.628979,...,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976,0.096391,0.0,0.0,0.356274
min,0.000000,1893.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
25%,2828.250000,1959.000000,35303.000000,0.000000,0.000000,24.000000,23.750000,1.000000,16.000000,3.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
50%,5458.500000,1970.000000,51381.500000,0.000000,0.000000,49.000000,173.500000,8.000000,67.000000,12.000000,...,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
75%,8427.750000,1977.000000,68522.000000,1.000000,1.000000,74.000000,504.250000,33.000000,232.000000,50.000000,...,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
max,11191.000000,1996.000000,666666.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,...,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.0,11.0,1.000000


We're gonna change the Year_Birth column to age because it's not as clear as age. We'll calculate the age based off the year of the most recent customer visit, which is the column Dt_Customer.

In [191]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])
df.Dt_Customer.max()


Timestamp('2014-12-06 00:00:00')

We converted our dt_customer so it can be read as an acutal date and return the correct most recent date. So we will use the year 2014 to calculate the age of our customers.

In [192]:
df['Year_Birth'] =2014 - df.Year_Birth
df = df.rename(columns = {'Year_Birth':'Age'})

Now that we've finished converting ages, we will convert people who have their education listed as 2n Cycle to Masters because a 2nd cycle is equivalent to a masters in other countries. 

In [193]:
df.loc[df.Education == '2n Cycle', 'Education' ] = 'Master'

The last thing we'll do is cleanup people's marital status

In [194]:
df.groupby('Marital_Status').size()

Marital_Status
Absurd        2
Alone         3
Divorced    232
Married     864
Single      480
Together    580
Widow        77
YOLO          2
dtype: int64

We'll merge Absurd, Alone, and Yolo with single since there all the same thing, but we'll keep divorced, widow, and single separate because these are very distinct types of relationships.

In [195]:
def marital_merge(row):
    if row.Marital_Status in {'Absurd', 'YOLO', 'Alone'}:
        row.Marital_Status = 'Single'
        return row
df.apply(marital_merge, axis = 'columns').



    

0       None
1       None
2       None
3       None
4       None
        ... 
2235    None
2236    None
2237    None
2238    None
2239    None
Length: 2240, dtype: object

Now that we have our data a little more readable, we're first going to discover how education might affect how successfull a campaing is. So we'll first group our data by education. 

In [196]:
df[df.Education.isin[]]


SyntaxError: invalid syntax (<ipython-input-196-8294025ade5b>, line 1)

In [ ]:
education.iloc[:, 15:20]